# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**


**Clustering** **Consulting** **Project**

In [224]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [258]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

In [259]:
spark = SparkSession \
    .builder \
    .appName("Spark ML example on clustering consulting project ") \
    .getOrCreate()

In [260]:
spark

In [261]:
data_path ="/content/hack_data.csv"

In [262]:
clustering_df = spark.read.csv(data_path,header = 'True',inferSchema='True')

In [263]:
clustering_df.columns



['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [264]:
clustering_df.dtypes



[('Session_Connection_Time', 'double'),
 ('Bytes Transferred', 'double'),
 ('Kali_Trace_Used', 'int'),
 ('Servers_Corrupted', 'double'),
 ('Pages_Corrupted', 'double'),
 ('Location', 'string'),
 ('WPM_Typing_Speed', 'double')]

In [265]:
clustering_df.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [266]:
clustering_df.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [267]:
clustering_df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [268]:
# This function use to print feature with null values and null count 
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [269]:
null_value_count(clustering_df)

[]

In [270]:
clustering_df.select("Location").distinct().show()

+--------------------+
|            Location|
+--------------------+
|            Anguilla|
|            Paraguay|
|               Macao|
|Heard Island and ...|
|               Yemen|
|             Tokelau|
|              Sweden|
|French Southern T...|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Malaysia|
|           Singapore|
|United States Vir...|
|              Turkey|
|      Western Sahara|
|              Malawi|
|                Iraq|
|Northern Mariana ...|
|             Germany|
+--------------------+
only showing top 20 rows



In [271]:
df=clustering_df.drop("Location")

In [272]:
df

DataFrame[Session_Connection_Time: double, Bytes Transferred: double, Kali_Trace_Used: int, Servers_Corrupted: double, Pages_Corrupted: double, WPM_Typing_Speed: double]

In [273]:
df.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+------------------+
|  count|                    334|               334|               334|              334|               334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|              40.0|
|    max|                   60.0|            1330.5|                 1|             10.0|              1

In [274]:
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [275]:

#feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             #'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']

In [276]:
#vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [245]:
#final_data = vec_assembler.transform(df)

In [277]:

from pyspark.ml.feature import StandardScaler


In [278]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)


In [279]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [280]:
# Normalize each feature to have unit standard deviation.
cluster_final_data = scalerModel.transform(final_data)


In [281]:
from pyspark.ml.clustering import KMeans

In [251]:
#kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
#kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [252]:
#model_k3 = kmeans3.fit(cluster_final_data)
#model_k2 = kmeans2.fit(cluster_final_data)

In [253]:
#wssse_k3 = model_k3.computeCost(cluster_final_data)
#wssse_k2 = model_k2.computeCost(cluster_final_data)

In [282]:
def meanScore(k,df):
  inputCol = df.columns[:]
  assembler = VectorAssembler(inputCols=inputCols,outputCol="features")
  kmeans = KMeans().setK(k)
  pipeModel2 = Pipeline(stages=[vec_assembler,kmeans])
  kmeansModel = pipeModel2.fit(df).stages[-1]
  kmeansModel.computeCost(assembler.transform(df))/data.count()

In [283]:
for k in range(2,3):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(cluster_final_data)
   

In [284]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   88|
|         2|   79|
|         0|  167|
+----------+-----+



In [285]:

model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

